In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn as sklearn
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel, SelectKBest
from sklearn.feature_selection import VarianceThreshold
from sklearn.cluster import DBSCAN
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.svm import SVC
from sklearn.metrics import balanced_accuracy_score
from sklearn.ensemble import BaggingClassifier
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.neighbors import LocalOutlierFactor
from sklearn.utils.class_weight import compute_class_weight

In [8]:
xtrain = pd.read_csv("X_train.csv")
xtrain = xtrain.iloc[:, 1:]
xtest = pd.read_csv("X_test.csv")
xtest = xtest.iloc[:, 1:]
ytrain = pd.read_csv("Y_train.csv")

# Remove id from ytrain
ytrain = ytrain.drop('id', axis = 1)
ytrain.head()

,y
0,1
1,0
2,1
3,1
4,1


In [ ]:
# Simple pipeline with grid search for SVM 

steps = [("scaler", preprocessing.StandardScaler()), ("classifier", SVC())]

pipeline = Pipeline(steps = steps)

parameters = {"classifier__kernel": ["rbf"],
              "classifier__gamma": ["auto"],
              "classifier__C": [0.1, 0.5, 1, 5, 10, 50, 100],
              "classifier__degree": [1, 2, 3, 4, 5, 6],
              "classifier__class_weight": ["balanced"]
             }

grid = GridSearchCV(pipeline, parameters, cv = 5, scoring = 'balanced_accuracy')

grid.fit(xtrain.values, ytrain.values.ravel())

In [11]:
# Simple pipeline with grid search for XGB

steps = [("scaler", preprocessing.StandardScaler()), ("classifier", xgb.XGBClassifier())]

pipeline = Pipeline(steps = steps)

parameters = {
        'classifier__min_child_weight': [1, 5, 10],
        'classifier__gamma': [0.5, 1, 1.5, 2, 5],
        'classifier__subsample': [0.6, 0.8, 1.0],
        'classifier__colsample_bytree': [0.6, 0.8, 1.0],
        'classifier__max_depth': [3, 4, 5]
        }

grid = GridSearchCV(pipeline, parameters, cv = 5, scoring = 'balanced_accuracy')

grid.fit(xtrain.values, ytrain.values.ravel(), classifier__class_weight={0:0.95, 1:0.05})

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


TypeError: fit() got an unexpected keyword argument 'class_weight'

In [10]:
print(grid.best_score_)
print(grid.best_params_)

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'

In [ ]:
# Submit model


xtrain = pd.read_csv("X_train.csv")
xtrain = xtrain.iloc[:, 1:]
xtest = pd.read_csv("X_test.csv")
xtest = xtest.iloc[:, 1:]
ytrain = pd.read_csv("Y_train.csv")

# Remove id from ytrain
ytrain = ytrain.drop('id', axis = 1)
ytrain.head()

'''
# Handle class imbalance with smote function
sm = SMOTE(random_state=42)
x_tmp, y_tmp = sm.fit_resample(xtrain.values, ytrain.values.ravel()) #x_train and y_train are now arrays
xtrain = pd.DataFrame(x_tmp, columns = xtrain.columns)
ytrain = pd.DataFrame(y_tmp, columns = ytrain.columns)
'''

# Scale the data (should this be done for each bootstrap sample? in that case how)
scaler = preprocessing.StandardScaler()
xtrain_scaled = scaler.fit_transform(xtrain)
xtrain = pd.DataFrame(xtrain_scaled, columns = xtrain.columns)
xtest_scaled = scaler.fit_transform(xtest)
xtest = pd.DataFrame(xtest_scaled, columns = xtest.columns)

'''
# Try K best
sel = SelectKBest(k=700)
sel.fit(xtrain, ytrain.values.ravel())
selected_feat = xtrain.columns[(sel.get_support())]
xtrain = xtrain.loc[:,selected_feat]
xtest = xtest.loc[:,selected_feat]
'''

# Model to fit
estimator = SVC(C = 0.5, gamma = 'auto', class_weight = 'balanced', degree = 1, kernel = 'rbf')
#estimator = xgb.XGBClassifier(objective="multi:softprob", random_state=40)

estimator.fit(xtrain, ytrain.values.ravel())
pred = estimator.predict(xtest)

index = pd.read_csv("sample.csv")
index['y'] = pred

index.to_csv("predictionsSimpleGridCVSVC.csv")